<h1>An elevator built with object-oriented programming.</h1>
<h4>Blake Rayvid - <a href=https://github.com/brayvid>https://github.com/brayvid</a></h4>

In [ ]:
class Building:
  """
  Attributes:
  - floors: list of Floors (0: Lobby, 1: MiddleFloor,..., -1: Roof)
  - elevators: list of Elevators
  - occupants: list of Riders in the building
  """
  def __init__(self, num_floors, num_elevators):
    """
    - Constructor parameters:
    - num_floors: Int > 1
    - num_elevators: Int > 0
    """
    if num_floors < 2:
      raise Exception("Building must have at least 2 floors.")
    elif num_elevators < 1:
      raise Exception("Building must have at least 1 elevator.")

    self.num_floors = num_floors
    self.num_elevators = num_elevators
    self.occupants = []

    # Instantiate floors of building
    self.floors = [Lobby(floor_number = 0, building = self)]
    for i in range(1, num_floors-1):
      self.floors.append(MiddleFloor(floor_number = i, building = self))
    self.floors.append(Roof(floor_number = num_floors-1, building = self))

    # Instantiate elevators of building
    self.elevators = []
    for i in range(num_elevators):
      self.elevators.append(Elevator(building = self))


class Floor:
  """
  Parent of Lobby, Roof and MiddleFloor. Not itself instantiated.

  Attributes:
  - waiting: list of Riders waiting on that floor
  - floor_number: Int
  - building: the containing Building object
  """
  def __init__(self, floor_number, building):
    """
    Constructor parameters:
    - floor_number: Int >= 0
    - building: a Building object
    """
    if floor_number < 0:
      raise Exception("'floor_number' must be a non-negative integer.")
    self.floor_number = floor_number
    self.building = building
    self.waiting = []


class Lobby(Floor):
  """
  Called by Building() only.

  Attributes:
  - waiting: list of Riders waiting on that floor
  - floor_number: Int
  - building: the containing Building object
  - up_button: UpDownButton(up = True)
  """
  def __init__(self, floor_number, building):
    """
    Constructor parameters:
    - floor_number: Int >= 0
    - building: a Building object
    """
    self.up_button = UpDownButton(up = True)
    super().__init__(floor_number = floor_number, building = building)


class Roof(Floor):
  """
  Called by Building() only.

  Attributes:
  - waiting: list of Riders waiting on that floor
  - floor_number: Int
  - building: the containing Building object
  - down_button: UpDownButton(up = False)
  """
  def __init__(self, floor_number, building):
    """
    Constructor parameters:
    - floor_number: Int >= 0
    - building: a Building object
    """
    self.down_button = UpDownButton(up = False)
    super().__init__(floor_number = floor_number, building = building)


class MiddleFloor(Floor):
  """
  Called by Building() only.

  Attributes:
  - waiting: list of Riders waiting on that floor
  - floor_number: Int
  - building: the containing Building object
  - up_button: UpDownButton(up = True)
  - down_button: UpDownButton(up = False)
  """
  def __init__(self, floor_number, building):
    """
    Constructor parameters:
    - floor_number: Int >= 0
    - building: a Building object
    """
    self.up_button = UpDownButton(up = True)
    self.down_buton = UpDownButton(up = False)
    super().__init__(floor_number = floor_number, building = building)


class Elevator:
  """
  Called by Building() only.

  Attributes:
  - weight_limit: maximum weight elevator can allow
  - current_weight: total weight of all occupants inside elevator
  - current_floor: what floor the elevator is currently at
  - building: the containing Building object
  - occupants: list of Riders inside elevator
  - direction: +1 = up, 0 = stopped, -1 = down
  - buttons: list of DestButtons inside elevator

  Methods:
  - ascend(): Go up one floor
  - descend(): Go down one floor
  """
  def __init__(self, building, weight_limit = 2000):
    """
    Constructor parameters:
    - building: what Building the elevator is in
    - weight_limit: [optional] maximum weight elevator allows before restricting entry. Default 2000.
    """
    self.weight_limit = weight_limit
    self.building = building
    self.current_floor = 0
    self.current_weight = 0
    self.occupants = []
    self.direction = 0 # not yet in use

    # Not yet in use
    self.buttons = []
    for i in range(self.building.num_floors):
      self.buttons.append(DestButton(i))

  # Go up one floor
  def ascend(self):
    if self.current_floor < self.building.num_floors-1:
      self.current_floor += 1
    else:
      raise Exception('Already at roof.')

  # Go down one floor
  def descend(self):
    if self.current_floor > 0:
      self.current_floor -= 1
    else:
      raise Exception('Already at lobby')


class Rider:
  """
  Attributes:
  - weight: weight of rider
  - origin_floor: what floor the rider is currently on before riding elevator
  - dest_floor: what floor the rider intends to travel to.
  - building: the containing Building object
  - elevator: the elevator number of the rider if in one, else None

  Methods:
  - enter(elevator_num): Have rider enter a specific elevator
  - exit_elev(): Have rider exit their current elevator at current floor
  - leave(): Have rider exit building (only if in lobby)
  - choose(floor_number): Have rider pick a destination floor
  """
  def __init__(self, building, weight = 150, dest_floor = 0):
    """
    Constructor parameters:
    - building: a Building object
    - weight: [optional] weight of rider. Default 150.
    - dest_floor: [optional] the floor the rider intends to travel to. Default 0.
    """
    self.weight = weight
    self.origin_floor = 0
    self.dest_floor = dest_floor
    self.building = building
    self.elevator = None
    self.building.floors[0].waiting.append(self)
    self.building.occupants.append(self)

  #
  def enter(self, elevator_num):
    if elevator_num > len(self.building.elevators) or elevator_num < 0:
      raise Exception("'Invalid 'elevator_num'.")

    if self.origin_floor != self.building.elevators[elevator_num].current_floor:
      raise Exception("Elevator is not at the rider's current floor.")

    if self.weight + self.building.elevators[elevator_num].current_weight > self.building.elevators[elevator_num].weight_limit:
      raise Exception("Not allowed - Elevator would be over weight limit.")

    waiting_idx = self.building.floors[self.origin_floor].waiting.index(self)
    self.building.floors[self.origin_floor].waiting.pop(waiting_idx)
    self.building.elevators[elevator_num].occupants.append(self)
    self.elevator = elevator_num
    self.building.elevators[elevator_num].current_weight += self.weight

  #
  def exit_elev(self):
    try:
      occupant_idx = self.building.elevators[self.elevator].occupants.index(self)
    except:
      raise Exception("Rider is not in an elevator.")

    self.building.elevators[self.elevator].occupants.pop(occupant_idx)
    self.building.floors[self.building.elevators[self.elevator].current_floor].waiting.append(self)
    self.origin_floor = self.building.elevators[self.elevator].current_floor
    self.building.elevators[self.elevator].current_weight -= self.weight
    self.elevator = None

  #
  def leave(self):
    if self.origin_floor == 0:
      waiting_idx = self.building.floors[0].waiting.index(self)
      self.building.floors[0].waiting.pop(waiting_idx)

      occupant_idx = self.building.occupants.index(self)
      self.building.occupants.pop(occupant_idx)
    else:
      raise Exception("Rider can only leave from the lobby.")

  #
  def choose(self, floor_number):
    self.dest_floor = floor_number


class Button:
  """
  Not fully implemented yet. Docstring pending.
  """
  def __init__(self):
    self.pressed = False

  #
  def push(self):
    self.pressed = True

  #
  def reset(self):
    self.pressed = False



class UpDownButton(Button):
  """
  Not fully implemented yet. Docstring pending.
  """
  def __init__(self, up):
    self.up = up
    super().__init__()


class DestButton(Button):
  """
  Not fully implemented yet. Docstring pending.
  """
  def __init__(self, floor_number):
    self.floor_number = floor_number
    super().__init__()

# Example automatic operation

In [ ]:
def sequence_runner(sequence):
  for step in sequence:
    exec(step)

floors = 20

# Script: one rider goes up to the roof and back down
seq = [f'bld = Building(num_floors = {floors}, num_elevators = 1)',
       'rider1 = Rider(building = bld)',
       'print([x.waiting for x in bld.floors])',
       'rider1.enter(0)',
       f'''for i in range({floors-1}):
            bld.elevators[0].ascend()
            rider1.exit_elev()
            print([x.waiting for x in bld.floors])
            rider1.enter(0)''',
       f'''for i in range({floors-1}):
            bld.elevators[0].descend()
            rider1.exit_elev()
            print([x.waiting for x in bld.floors])
            rider1.enter(0)''']

sequence_runner(seq)

[[<__main__.Rider object at 0x7bc5d172f2e0>], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [<__main__.Rider object at 0x7bc5d172f2e0>], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [<__main__.Rider object at 0x7bc5d172f2e0>], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [<__main__.Rider object at 0x7bc5d172f2e0>], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [<__main__.Rider object at 0x7bc5d172f2e0>], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [<__main__.Rider object at 0x7bc5d172f2e0>], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [<__main__.Rider object at 0x7bc5d172f2e0>], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [<__main__.Rider object at 0x7bc5d172f2e0>], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [], [],

# Example manual operation

In [ ]:
# ACTION: Create a new building object with 10 floors and 3 elevators
bld = Building(num_floors = 10, num_elevators = 3)

In [ ]:
# Display what floor each elevator is currently on
print([x.current_floor for x in bld.elevators])

[0, 0, 0]


In [ ]:
# Display the building's list of floors
bld.floors

In [ ]:
# Display the riders waiting for the elevator at each floor
print([x.waiting for x in bld.floors])

[[], [], [], [], [], [], [], [], [], []]


In [ ]:
# Display the riders in each elevator
print([x.occupants for x in bld.elevators])

[[], [], []]


In [ ]:
# Display all occupants of the building
bld.occupants

[]

In [ ]:
# ACTION: Create a new rider associated with the building (they automatically enter the building at the lobby)
rider1 = Rider(building = bld)
rider1

In [ ]:
# Display the building occupants again
bld.occupants

In [ ]:
# Display the riders waiting for the elevator at each floor
print([x.waiting for x in bld.floors])

[[<__main__.Rider object at 0x7bc5d1760070>], [], [], [], [], [], [], [], [], []]


In [ ]:
# ACTION: Have the rider enter the first elevator
rider1.enter(0)

In [ ]:
# Display the occupants of that elevator
bld.elevators[0].occupants

In [ ]:
# Display riders now waiting at the lobby
bld.floors[0].waiting

[]

In [ ]:
# ACTION: Go up to the roof
for i in range(9):
  bld.elevators[0].ascend()

In [ ]:
# Display what floor each elevator is currently on
print([x.current_floor for x in bld.elevators])

[9, 0, 0]


In [ ]:
# ACTION: Have the rider exit the elevator at the roof
rider1.exit_elev()

In [ ]:
# Display the riders waiting for the elevator at each floor
print([x.waiting for x in bld.floors])

[[], [], [], [], [], [], [], [], [], [<__main__.Rider object at 0x7bc5d1760070>]]


In [ ]:
# ACTION: Have the rider enter the elevator again from the roof
rider1.enter(0)

In [ ]:
# ACTION: Go back down to the lobby
for i in range(9):
  bld.elevators[0].descend()

In [ ]:
# Display what floor each elevator is currently on
print([x.current_floor for x in bld.elevators])

[0, 0, 0]


In [ ]:
# ACTION: Have the rider exit the elevator at the lobby
rider1.exit_elev()

In [ ]:
# Display the riders waiting for the elevator at each floor
print([x.waiting for x in bld.floors])

[[<__main__.Rider object at 0x7bc5d1760070>], [], [], [], [], [], [], [], [], []]


In [ ]:
# Display the occupants of the elevator
bld.elevators[0].occupants

[]

In [ ]:
# Display the occupants of the building
bld.occupants

In [ ]:
# ACTION: Have the rider exit the building
rider1.leave()

In [ ]:
# Display building occupants
bld.occupants

[]

# To do

- Elevator weight limit ✔
- Floor buttons inside elevator - can select any/all of them at any time ✔
- Status: the floor the elevator is currently at ✔
- Status: going up, going down or stopped ✔
- Floors - top floor only has down button, bottom floor only has up button, all intermediate floors have both ✔
- Passengers with weights and desired destination floor ✔
- Passengers can enter and exit elevator at any floor ✔
<h4><u>Button functionality</u></h4>
- Pressing up/down buttons on a floor has to call an elevator to that floor
- Pressing floor buttons inside the elevator has to make the elevator go to that floor
- What if origin floor and destination floor are same thing (person presses button to floor they are on)? Nothing should happen.
- When is button state reset?
- Emergency stop toggle
<h4><u>Advanced automation</u></h4>
- Queue of floors to visit
- Decider algorithm - which floor to go to next based on which buttons have been pressed and in what order
- Some kind of sequence of discrete states to keep track of when buttons were pressed in relation to which floor the elevator is currently on
- Keep track of the elevator direction and do not reverse direction until there are no more stops in the current direction
<h4><u>Extensions</u></h4>
- Elevators can only go to a predefined set of floors, and they cover different ranges.
